<a href="https://colab.research.google.com/github/udothemath/ncku_customer_embedding/blob/main/WenMing/2_Use_embedding_do_user_profile_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 如果有使用 coloab 再執行此 cell
from google.colab import drive
drive.mount('/gdrive')
!ln -s /gdrive/MyDrive/colab/NCKU_embedding/data/ ./data

from google.colab import auth
auth.authenticate_user()
from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'neat-motif-123006'
client = bigquery.Client(project=project_id)


Mounted at /gdrive


# Load Common Trainig Data and Testing Data

In [ ]:
import os
import numpy as np
import pandas as pd
from time import time
from tqdm.notebook import tqdm
from sklearn.preprocessing import MinMaxScaler


import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, auc, roc_auc_score,f1_score

import pickle

## 整理出 User Profile 的 Prediciton Target Label
* 有部份的使用者婚姻狀態有改變，所以要先去除掉


In [ ]:
result = client.query("""
select chid, masts from `neat-motif-123006.crd.x_train`
where chid in (
SELECT chid FROM `neat-motif-123006.crd.x_train`
where masts = 1 or masts = 3
group by chid having count(distinct masts) = 1)
group by chid, masts having masts=1 or masts=3 order by chid  
""")
valid_user_masts = result.to_dataframe()

In [ ]:
import pickle
with open("./data/sample_50k/valid_user_masts.pkl","wb") as fw:
    pickle.dump(valid_user_masts,fw)

In [ ]:
valid_user_masts.masts.unique()

array([1, 3])

In [ ]:
valid_user_masts.masts =valid_user_masts.masts.map({1:0,3:1})

In [ ]:
valid_user_masts.masts.unique()

array([0, 1])

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
X = valid_user_masts.chid
y = valid_user_masts.masts

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

In [ ]:
train_y = valid_user_masts.masts.to_list()

# 使用 Pytroch Node2Vec

In [ ]:
fr = open("/content/data/sample_50k/embedding/Word2Vec_0221.pkl",'rb')
model = pickle.load(fr)
fr.close()

# 使用 Word2Vec and StellarGraph
*  logistic regresson classifier -> acc = 0.5737770649558942
*  dnn classifier -> acc = 0.5643544506816359

In [ ]:
import pickle

In [ ]:
# load word2vec id
fr = open("./data/sample_50k/embedding/user_and_mcc_idx_map.pkl","rb")
user_and_mcc_idx_map = pickle.load(fr)

In [ ]:
# user_and_mcc_idx_map
idx2_user_mcc = {}

for k in user_and_mcc_idx_map:
    v = user_and_mcc_idx_map[k]
    idx2_user_mcc[v] = k

In [ ]:
import numpy as np
user_idx_map = np.load('./data/sample_50k/sample_idx_map.npy', allow_pickle=True).item()

In [ ]:
# load word2vec model

fr = open("./data/sample_50k/embedding/Word2Vec_0221.pkl",'rb')
w2vec = pickle.load(fr)
fr.close()

In [ ]:
# 取 user'd embedding
user_embedding = w2vec.wv.vectors[:50000]

In [ ]:
user_embedding.shape

(50000, 128)

In [ ]:
# filter user 
user_embedding_train = user_embedding.take(X_train.to_list(), axis=0)

In [ ]:
user_embedding_test = user_embedding.take(X_test.to_list(), axis=0)
user_embedding_test.shape

(4988, 128)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(
    Cs=10, cv=10, scoring="accuracy", verbose=False, multi_class="ovr", max_iter=300
)
clf.fit(user_embedding_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=10, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=300, multi_class='ovr', n_jobs=None, penalty='l2',
                     random_state=None, refit=True, scoring='accuracy',
                     solver='lbfgs', tol=0.0001, verbose=False)

In [ ]:
y_pred = clf.predict(user_embedding_test)
accuracy_score(y_test,y_pred)

0.5737770649558942

In [ ]:
y_pred_proba = clf.predict_proba(user_embedding_test)
roc_auc_score(y_test,y_pred)

0.5040664877234571

In [ ]:
y_pred = clf.predict(user_embedding_test)

In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Dropout
from keras.optimizers import Adam
# from keras.metrics import categorical_crossentropy
from keras.losses import categorical_crossentropy

In [ ]:
simple_model = Sequential([
    Dense(user_embedding_train.shape[1], input_shape=(user_embedding_train.shape[1], )),
    Dropout(0.25),
    Dense(32, activation='relu'),
    Dropout(0.25),
    Dense(2, activation='softmax')
])

In [ ]:
simple_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 66        
Total params: 20,706
Trainable params: 20,706
Non-trainable params: 0
_________________________________________________________________


In [ ]:
simple_model.compile(Adam(lr=0.001),loss=categorical_crossentropy,metrics=['accuracy'])


In [ ]:
y_train

21410    1
42501    1
23692    1
17851    1
9720     0
        ..
11284    0
44732    0
38158    0
860      1
15795    0
Name: masts, Length: 44888, dtype: int64

In [ ]:
y_train=keras.utils.to_categorical(y_train)
y_train

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [ ]:
user_embedding_train.shape

(44888, 128)

In [ ]:
simple_model.fit(
    user_embedding_train, y_train, validation_split=0.2, batch_size=25, epochs=20, shuffle=True, verbose=2)


In [ ]:
y_pred = simple_model.predict_classes(user_embedding_test)

In [ ]:
y_pred_proba = simple_model.predict(user_embedding_test)

In [ ]:
accuracy_score(y_test,y_pred)

0.5643544506816359